# Computation and plotting of TWAS results
### Author: Mihail Mihov

#### ! Install FUSION TWAS and download the necessary 1000 Genomes LD reference data and GTEx whole blood eQTL data (weights) by following the instructions at 'http://gusevlab.org/projects/fusion/#installation' !

In [ ]:
#load the necessary libraries
library(data.table)
library(tidyverse)
library(ggplot2)

#read in the GWAS summary statistics generated prior to TWAS analysis
sumstats=fread("~Your_directory/GWAS_all.sumstats",data.table=F)
#extract only the columns containing SNP names (SNP), first allele (A1), second allele (A2) and the Z scores (Z) 
sumstats=sumstats[,c("SNP","A1","A2","Z")]

In [ ]:
#create a list of all GTEx weigths for each tissue
tissues=list.files("~Your_directory/fusion_twas-master/WEIGHTS/WEIGHTS", pattern="\\.pos")

In [ ]:
#set output directory for the TWAS results. This will be the foulder containing results foulders for each tissue.
out_dir="~Your_directory/TWAS/"

#loop over all tissues in the GTEx database
for (tissue in tissues[1:length(tissues)]){
    
    message(paste0("----------------- TISSUE: ", tissue, " (", which(tissue==tissues), "/", length(tissues), ") ------------------"))
    ## then loop over all chromosomes
    for (chr in 1:22){
        message(paste0("----------------- CHROMOSOME: ", chr, "/22 ------------------"))
        # construct the R command to be executed in terminal
        command=paste0("Rscript ~Your_directory/fusion_twas-master/FUSION.assoc_test.R ",
                   "--sumstats ", sumstats, # this is the path to the sumstats file
                   " --weights ~Your_directory/fusion_twas-master/WEIGHTS/WEIGHTS/", tissue, # here is the tissue from the loop
                   " --weights_dir ~Your_directory/fusion_twas-master/WEIGHTS/WEIGHTS/", # static directory of weights
                   " --ref_ld_chr ~Your_directory/fusion_twas-master/LDREF/1000G.EUR.", # static directory of LD data
                   " --chr ", chr, # here is the chromosome from the loop
                   " --out ",out_dir,"TWAS_chr", chr, "_", gsub("\\.pos", "", tissue),"_results.txt")
        # run the command
       system(command)
   }
}

In [ ]:
#Combine the results from each tissue into one table
result_files=list.files("~Your_directory/TWAS/", pattern="_results.txt", full.names =T, recursive=TRUE)
results = lapply(result_files, fread, data.table=F)
results_data = data.frame(rbindlist( results ))

In [ ]:
#Add FDR column to be used for extra quality control
results_data$FDR=p.adjust(results_data$TWAS.P, method="fdr")

#restrict the results to only coding genes to be ploted
results_data$ENSG=gsub("\\..*", "", gsub(".*\\.ENSG","ENSG", results_data$FILE))

genes=fread("~Your_dir/coding_genes.tsv", data.table=F)
colnames(genes)[1]="ID"
genes$ID=gsub("\\..*", "", genes$ID)
genes=genes$ID
head(genes,1)

plot=results_data[which(results_data$ENSG %in% genes),c("ID","TWAS.Z","FDR","ENSG")]

#Restrict significant results to instances where FDR <= 0.01 and absolute Z-score > 5.5 (i.e. P-val < 10^(-8))
plot=plot[which(complete.cases(plot$FDR)),]
plot=plot[which(plot$FDR<=0.01),]
plot$absZ=abs(plot$TWAS.Z)
plot=plot[order(plot$absZ, decreasing=T),]
plot=plot[!duplicated(plot$ID),]
plot=plot[plot$absZ>5.5,]
colnames(plot)[1]="PLOT"

#Plot the results using 'ggplot2'

results_data=left_join(by=c("ENSG","TWAS.Z","FDR"), results_data, plot)
head(results_data)


plot_results=results_data[order(results_data$CHR,results_data$P0 ), c("ID","CHR","P0","TWAS.Z","PLOT")]
plot_results=plot_results[!is.na(plot_results$TWAS.Z ),]
plot_results$seq=1:nrow(plot_results)
plot_results$basecolour="grey"
plot_results[(plot_results$CHR%%2 )==1,"basecolour"]="black"
plot_results$symbol="1"
plot_results[(plot_results$ TWAS.Z)>5.5,"symbol"]="25"
plot_results[(plot_results$ TWAS.Z)< -5.5,"symbol"]="26"
plot_results$symbol=factor(plot_results$symbol)
plot_results$ID=as.character(plot_results$ID)

labels = aggregate(plot_results$seq, by = list(plot_results$CHR),mean)
labels$tick=labels$x
labels$labels=unique(plot_results$CHR)

options(repr.plot.width=8, repr.plot.height=4, repr.plot.res=240)

ggplot(plot_results, aes(y=TWAS.Z, x=seq)) +
geom_hline(yintercept = -5.5,colour="red", linetype = "longdash")+
geom_hline(yintercept = 5.5,colour="red", linetype = "longdash")+
#geom_point(aes(color =basecolour , fill=basecolour, size=abs(TWAS.Z), shape=factor(ifelse(TWAS.Z>0, 2, 6))))+
geom_point(aes(color =basecolour , fill=basecolour, size=abs(TWAS.Z), shape=symbol))+

scale_shape_manual(values=c(16, 24, 25))+ theme_classic() +
scale_color_manual(values=c( "gray33","grey"))+ scale_fill_manual(values=c( "gray33","grey"))+
scale_size_continuous(range = c(0,3), breaks=c(0,1,2,3,4,5,6,7,8,9)) +
theme(legend.position = "none", axis.text.x=element_text(angle=45,hjust=1))+
geom_hline(yintercept = 0, col="black")+
xlab("Chromosomes") +ylab("Z-Score")+
scale_x_continuous(breaks = labels$tick, labels = labels$label, expand = c(0.01, 0))+
ggrepel::geom_text_repel(aes(label=ifelse(abs(TWAS.Z)>5.5,as.character(PLOT),'')), size=3,angle=0,max.overlaps=40,nudge_x=1000,force=5)